## Building a Scalable Agentic Solution with Object-Oriented Principles

Designing scalable LLM solutions requires more than just raw intelligence of the models — it also demands application code be easy to maintain, extend, and modify as requirements evolve. By leveraging Object-Oriented Programming (OOP) principles, we can create modular, testable systems that will stand the test of time. This Cookbook walks through an OOP-driven Application built on OpenAI’s Chat Completions API, using design techniques such as inheritance, interfaces, dependency injection, and the factory pattern.

### Overview of Core Components

The solution is provided in the directory `./object_oriented_agents/` you can examine the code to follow along. For brevity, this Cookbook will focus on the overview of the architecture and a concrete implementation for demonstration purpose. The code is organized into the following directories:

1. Core Classes: Form the basis of the application and provide a common interface for all agents, tools, and messages. 
- **BaseAgent**: This is the base class for all agents. It provides a common interface for all agents to follow.
- **ToolInterface**: This is the interface for all tools. It provides a common interface for all tools to follow.
- **ToolManager**: This is the manager for all tools. It provides a common interface for all tools to follow.
- **ChatMessages**: This is the manager for all chat messages. It provides a common interface for all chat messages to follow.

2. Services: Provide a common interface for language models. A language model interface is required for the agent to invoke LLM API. 
- **LanguageModelInterface**: This is the interface for all language models. It provides a common interface for all language models to follow.
- **OpenAILanguageModel**: This is a concrete implementation of the LanguageModelInterface that uses the OpenAI Chat Completions API.

3. Utils:
- **Logger**: This is the logger for the application. It provides a common interface for all logging to follow.
- **OpenAIClientFactory**: This is the factory for the OpenAI client. It returns an instance of the OpenAI client. 

### Key Associations and Dependencies
- BaseAgent, ToolInterface, and LanguageModelInterface are abstract classes meant to be extended for specific use cases.
- BaseAgent composes a ChatMessages instance for managing conversation logs (i.e., storing and retrieving messages).
- BaseAgent optionally holds a ToolManager but always requires a LanguageModelInterface.
- ToolManager maintains a set of ToolInterface implementations, uses ChatMessages for context, and delegates API calls to the language model.
- OpenAILanguageModel relies on OpenAIClientFactory for client creation, ensuring a clean separation of concerns around API-key resolution and client configuration.

The following diagram represents the main classes, interfaces and their relationship.

![UML Diagram](./resources/diagrams/images/class_diagram_mermaid.png)

**Prerequisites**:

We will use the WeatherAgent as an example to demonstrate the implementation of the object_oriented_agents. Make sure : 

1. Append the path to the `resources/object_oriented_agents/` directory to the Python path so that we you can import the classes. 
2. Obtain an API key for the OpenAI. Set it in the environment variable `OPENAI_API_KEY`. 
3. Obtain an API key for the OpenWeather API. Set it in the environment variable `OPENWEATHER_API_KEY`. 


In [12]:
import sys, os

# Add the parent directory of 'core_classes' to the Python path
sys.path.append(os.path.abspath('resources/object_oriented_agents/'))

### Object-Oriented Approach to Build a Weather Agent with Tool Calling

Using the object-oriented approach, we will build a Weather agent that can fetch and display weather given a zipcode. Its a three step process as follows:

#### Step 1: Define the Tool

Define the **CheckWeatherTool**, which is a concrete implementation of the **ToolInterface**. The interface ensures tools are defined in a consistent manner. 

**ToolInterface** requires concrete implementations to: 
1. Define the `get_definition()` method, which returns the JSON/dict definition of this tool's function. Conforms to OpenAI function calling specifications. 
2. Define the `run()` method, which executes the tool using the provided arguments and returns the result as a string.  

The tool definition for the LLM tool call and the implementation of the tool execution is wrapped in the same class for maintainability. In step 2, we will use the **ToolManager** class to register the tool with the agent. Note that once a tool such as **CheckWeatherTool** is defined, it can be registered with multiple agents to use. 

In [13]:
import requests
from typing import Dict, Any
from resources.object_oriented_agents.core_classes.tool_interface import ToolInterface

class CheckWeatherTool(ToolInterface):
    """
    A tool to check the latest weather given a zipcode.
    """

    def __init__(self, openweather_api_key: str = None):
        # Pass the API key for the weather service 
        self.openweather_api_key = openweather_api_key

    def get_definition(self) -> Dict[str, Any]:
        """
        Return the JSON/dict definition of this tool's function.
        Conforms to OpenAI function calling specifications.
        """
        return {
            "function": {
                "name": "check_weather",
                "description": "Check the current weather for a given US zipcode.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zipcode": {
                            "type": "string",
                            "description": "The 5-digit US ZIP code for which to check the weather."
                        }
                    },
                    "required": ["zipcode"]
                }
            }
        }

    def run(self, arguments: Dict[str, Any]) -> str:
        """
        Execute the tool using the provided arguments and return a result as a string.
        Here, we'll call the OpenWeather API (if you have a key) or just return a mock result.
        """
        zipcode = arguments.get("zipcode")
        if not zipcode:
            return "Error: No zipcode provided."

        if not self.openweather_api_key:
            raise ValueError("Error: No API key provided for weather service.")
        try:
            # Example call to OpenWeatherMap (if you have an API key and want real data).
            # Adjust the endpoint and parameters as needed for your chosen weather service.
            url = f"https://api.openweathermap.org/data/2.5/weather"
            params = {
                "zip": zipcode,
                "appid": self.openweather_api_key,
                "units": "imperial"
            }
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            # Extracting simple weather info
            city_name = data.get("name", "Unknown location")
            main_weather = data["weather"][0]["description"]
            temp = data["main"]["temp"]

            return f"The current weather in {city_name} ({zipcode}): {main_weather}, {temp}°F."
        except Exception as e:
            return f"Error when calling weather API: {str(e)}"

### Step 2: Define the Agent

Next, define the **WeatherAgent** that uses the **CheckWeatherTool**. Here are the steps to define the agent:

- Define the Agent: **WeatherAgent** class that extends the **BaseAgent** class. You can choose to pass the **developer_prompt** and **model_name** to the **BaseAgent** class in the constructor.  
- Setup the tools: Define the **setup_tools()** method to register the **CheckWeatherTool** with the **ToolManager**.

In addition to the tools, the Agent also requires a **LanguageModelInterface** to invoke the LLM API.  

- Setup the language model: Define the **language_model_interface** to use the **OpenAILanguageModel** class. This is a concrete implementation of the **LanguageModelInterface** that uses the OpenAI Chat Completions API.

Additionally, we provide the agent ability to use a logger. The developer can pass the logger to the **BaseAgent** class in the constructor. If not provided, the agent will use a default logger. 

- Define the **logger** to use the **get_logger** function from the **logger** module. This is a utility function that returns a logger instance with the specified name and level. Logging level can be set to DEBUG for verbose output as the Agent performs its tasks. 


In [14]:
# object_oriented_agents/agents/weather_agent.py

from typing import Optional
import logging

from dotenv import load_dotenv
import os

from resources.object_oriented_agents.core_classes.base_agent import BaseAgent
from resources.object_oriented_agents.core_classes.tool_manager import ToolManager
from resources.object_oriented_agents.services.language_model_interface import LanguageModelInterface
from resources.object_oriented_agents.services.openai_language_model import OpenAILanguageModel
from resources.object_oriented_agents.utils.logger import get_logger

# Set the verbosity level: DEBUG for verbose output, INFO for normal output. You could create a logger for each agent to keep the logs organized. 
app_logger = get_logger("WeatherAgent", level=logging.DEBUG)


# Create a LanguageModelInterface instance using the OpenAILanguageModel
language_model_api_interface = OpenAILanguageModel(api_key=os.getenv("OPENAI_API_KEY"), logger=app_logger)

load_dotenv()
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")  

class WeatherAgent(BaseAgent):
    """
    An agent that can check the weather by calling the CheckWeatherTool.
    """

    def __init__(
        self,
        developer_prompt: str = "You are a Weather Agent. You answer questions about the weather.",
        model_name: str = "gpt-4o",
        logger= app_logger,
        language_model_interface: Optional[LanguageModelInterface] = language_model_api_interface,
        openweather_api_key: Optional[str] = openweather_api_key,
    ):
        super().__init__(developer_prompt, model_name, logger, language_model_interface)
        # Create a tool manager specifically for this agent
        self.tool_manager = ToolManager(logger=self.logger, language_model_interface=self.language_model_interface)
        self.openweather_api_key = openweather_api_key
        # Set up the weather tool
        self.setup_tools()

    def setup_tools(self) -> None:
        """
        Register all the tools needed by this agent, e.g., the weather tool.
        """
        weather_tool = CheckWeatherTool(openweather_api_key=self.openweather_api_key)
        self.tool_manager.register_tool(weather_tool)

### Step 3: Create the Concrete Agent

Now that we have defined the agent, we can create an instance of the agent and use it to handle a user's task. In setting up the agent, we have abstracted out a lot of the details of the implementation. The application code it self is clean and simple, and does not need to know about the underlying implementation details of the agent. 

In [15]:
# Create the WeatherAgent
agent = WeatherAgent()

# Example user input
user_question = "What is the weather in 90210 right now?"

# Have the agent handle the user's task
result = agent.task(user_question)

print("Agent response:", result)

2024-12-26 11:34:39,221 - WeatherAgent - DEBUG - Registered tool 'check_weather': {'function': {'name': 'check_weather', 'description': 'Check the current weather for a given US zipcode.', 'parameters': {'type': 'object', 'properties': {'zipcode': {'type': 'string', 'description': 'The 5-digit US ZIP code for which to check the weather.'}}, 'required': ['zipcode']}}}
2024-12-26 11:34:39,222 - WeatherAgent - DEBUG - Starting task: What is the weather in 90210 right now? (tool_call_enabled=True)
2024-12-26 11:34:39,222 - WeatherAgent - DEBUG - Adding user message: What is the weather in 90210 right now?
2024-12-26 11:34:39,222 - WeatherAgent - DEBUG - Tool definition retrieved for 'check_weather': {'name': 'check_weather', 'description': 'Check the current weather for a given US zipcode.', 'parameters': {'type': 'object', 'properties': {'zipcode': {'type': 'string', 'description': 'The 5-digit US ZIP code for which to check the weather.'}}, 'required': ['zipcode']}}
2024-12-26 11:34:39,2

Agent response: The weather in Beverly Hills (90210) right now is clear with a temperature of 62.91°F.


Let's pass another task to the agent. 

In [16]:
user_question = "What is the weather in 94551 right now?"

# Have the agent handle the user's task
result = agent.task(user_question)

print("Agent response:", result)

2024-12-26 11:34:41,017 - WeatherAgent - DEBUG - Starting task: What is the weather in 94551 right now? (tool_call_enabled=True)
2024-12-26 11:34:41,017 - WeatherAgent - DEBUG - Adding user message: What is the weather in 94551 right now?
2024-12-26 11:34:41,018 - WeatherAgent - DEBUG - Tool definition retrieved for 'check_weather': {'name': 'check_weather', 'description': 'Check the current weather for a given US zipcode.', 'parameters': {'type': 'object', 'properties': {'zipcode': {'type': 'string', 'description': 'The 5-digit US ZIP code for which to check the weather.'}}, 'required': ['zipcode']}}
2024-12-26 11:34:41,018 - WeatherAgent - DEBUG - Tools available: [{'type': 'function', 'function': {'name': 'check_weather', 'description': 'Check the current weather for a given US zipcode.', 'parameters': {'type': 'object', 'properties': {'zipcode': {'type': 'string', 'description': 'The 5-digit US ZIP code for which to check the weather.'}}, 'required': ['zipcode']}}}]
2024-12-26 11:3

Agent response: The weather in Livermore (94551) right now is overcast, with a temperature of 55.83°F.


### Key Design Principles for Object-Oriented Approach

1. Abstraction & Interfaces
- Abstract classes (BaseAgent, ToolInterface, LanguageModelInterface) define clear contracts, making it easy to swap in new implementations—such as different language models or tools—without disrupting existing code.
- This flexibility is key to building scalable, future-proof systems.

2. Encapsulation & Single Responsibility
- Each class is designed around a specific concern:
- ChatMessages manages message storage and manipulation.
- ToolManager handles tool registration, definitions, and invocation logic.
- BaseAgent orchestrates user tasks, message flow, and the interactions between the language model and tools.
- OpenAILanguageModel encapsulates direct communication with the OpenAI API.
- Following the Single Responsibility Principle (SRP) ensures that classes remain focused, maintainable, and easy to reason about.

3. Dendency Injection & Inversion
- Dependencies (e.g., LanguageModelInterface, logging utilities, ToolManager) are passed into constructors or provided through interfaces, preventing rigid coupling to concrete implementations.
- This design aligns with the Dependency Inversion Principle, where high-level modules (BaseAgent) depend on abstractions (LanguageModelInterface) rather than concrete classes (OpenAILanguageModel).

4. Factory Pattern
- OpenAIClientFactory centralizes client creation, including any logic for API keys, environment variables, or other configuration details.
- By encapsulating client instantiation, you can easily adjust how clients are created without scattering changes throughout the codebase.

5. Separation of Concerns
- Responsibilities for tool invocation, message handling, and external API communication are each isolated within their own classes.
- This separation yields a system that is straightforward to maintain, debug, and extend over time—an essential trait in scalable architectures.